In [5]:
## step 0: 训练数据准备, 准备用于训练的category下的sku数据列表, 结果保存为两个csv文件
#!python prepare_v0.py

In [6]:
!python prepare.py --split_idx 100

<< 总类目数量:  517
<< 总类目匹配数量:  100
<< 训练匹配数量:  100
<< 评估类目:  417
100%|████████████████████████████████████████| 417/417 [00:00<00:00, 500.64it/s]
Data written to json
Data written to json
train samples:498,test samples:56
Data written to json


## 模型训练

In [7]:
!pip install openpyxl accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 3.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.4 MB/s eta 0:00:0000:01


In [10]:
!accelerate launch \
train_Clip_ViT_fullfc.py --model_name 'openai/clip-vit-large-patch14-336' \
--train_data_file '../data/train/train_vit_pairwise.json' --test_data_file '../data/train/test_vit_pairwise.json' \
--image_dir '../data/images' --max_epoch 1 \
--batch_size 2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `8`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
[W Utils.hpp:133] Warning: Envi

## eval 

In [ ]:
%time
!python infer_Clip_ViT_fullfc.py \
--model_name 'vit_pairwise_model_emb/model.pth' \
--test_data_file '../data/train/val_vit.json' \
--image_dir '../data/images' 

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 6.44 μs
是否有GPU可用: True
GPU数量: 8
Namespace(model_name='vit_pairwise_model_emb/model.pth', in_channels=1032, train_data_file=None, test_data_file='../data/train/val_vit.json', image_dir='../data/images', batch_size=8, num_cls=3, max_epoch=1, lr=1e-05, weight_decay=0.005, num_workers=0, print_interval_steps=100, use_score_model=1, use_dropout=0, keep_prob=0.8, clip_value=1.0, save_path='vit_pairwise_model_emb', save_name=None)
模型加载用时: 2.3159291744232178
!!!!!! 1906
 95%|█████████████████████████████████████  | 1814/1906 [13:25<00:44,  2.07it/s]

In [ ]:
!python eval.py

## single infer

In [34]:
%%writefile local_infer_single.py

import torch
import os
from PIL import Image
import argparse
from transformers import CLIPVisionModel, CLIPImageProcessor, AutoTokenizer, AutoModel

def expand2square(pil_img, background_color):
    width, height = pil_img.size  # 获得图像宽高
    if width == height:  # 相等直接返回不用重搞
        return pil_img
    elif width > height:  # w大构建w尺寸图
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))  # w最大，以坐标x=0,y=(width - height) // 2位置粘贴原图
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result
    
class Pairwise_ViT_Infer(torch.nn.Module):
    def __init__(self, vision_tower, num_labels=2):
        super(Pairwise_ViT_Infer, self).__init__()

        self.vit = vision_tower
        self.score_layer = torch.nn.Linear(4096, 1)

    def forward(self, text1, text2, x1, x2):
        x1 = self.vit(x1, output_hidden_states=True)['last_hidden_state']
        x2 = self.vit(x2, output_hidden_states=True)['last_hidden_state']

        # print(x1[:, 0, :].shape)
        # print(text1.shape)
        feature1 = torch.concat([x1[:, 0, :], text1.squeeze(dim=1), x2[:, 0, :], text2.squeeze(dim=1)], dim=-1)
        # Use the embedding of [CLS] token
        output1 = self.score_layer(feature1)

        return output1
    
def init_model(model_path):
    ## load first-page-pic-scoring model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name=os.path.join(model_path)

    vit_image_processor = CLIPImageProcessor.from_pretrained('openai/clip-vit-large-patch14-336')  # 加载图像预处理
    vision_tower = CLIPVisionModel.from_pretrained('openai/clip-vit-large-patch14-336')  # 加载图像模型
    vit_model = Pairwise_ViT_Infer(vision_tower).to(device)
    vit_model.load_state_dict(torch.load(model_name, map_location=device), strict=True)

    tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5', model_max_length=512)
    emb_model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5')

    vit_image_processor = vit_image_processor
    vit_model = vit_model
    return tokenizer,emb_model,vit_image_processor,vit_model

###
def process_image(img_path,vit_image_processor,device):
    image1 = Image.open(img_path).convert("RGB")
    image1 = expand2square(image1, tuple(int(x * 255) for x in vit_image_processor.image_mean))
    image1 = vit_image_processor.preprocess(image1, return_tensors='pt')['pixel_values'][0].unsqueeze(0).to(device)    
    return image1

def get_inputs(tokenizer,emb_model,text1, text2, device):
        # Tokenize sentences
        encoded_input1 = tokenizer(text1, padding=True, truncation=True,
                                        return_tensors='pt')
        text_output1 = torch.nn.functional.normalize(emb_model(**encoded_input1)[0][:, 0], p=2, dim=1).to(device)

        encoded_input2 = tokenizer(text2, padding=True, truncation=True,
                                        return_tensors='pt')
        
        text_output2 = torch.nn.functional.normalize(emb_model(**encoded_input2)[0][:, 0], p=2, dim=1).to(device)

        return text_output1,text_output2

def main(model_path, img1, img2, text1, text2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  #  model_path = 'vit_pairwise_model_emb/model.pth'
    tokenizer,emb_model,vit_image_processor,vit_model = init_model(model_path)
    
    image1 = process_image(img1,vit_image_processor,device)
    image2 = process_image(img2,vit_image_processor,device) 

    text1, text2 = get_inputs(tokenizer,emb_model,text1, text2, device)

    score = vit_model(text1, text2, image1, image2)
    
    print(score)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, default='vit_pairwise_model_emb/model.pth')
    parser.add_argument('--image1', type=str)
    parser.add_argument('--text1', type=str)
    parser.add_argument('--image2', type=str)
    parser.add_argument('--text2', type=str)
    args = parser.parse_args()

    main(args.model_path, args.image1, args.text1, args.image2, args.text2)


Overwriting local_infer_single.py


In [36]:
%%time
!python local_infer_single.py --model_path 'vit_pairwise_model_emb/model.pth' \
--image1 '../data/images/41H+cMdoqqL._AC_US200_.jpg' \
--text1 '../data/images/61bf5e+XrAL._AC_US200_.jpg'\
--image2 'Gewerbe, Industrie & Wissenschaft:Materialtransport, Ladungssicherung & Zubehör:Zieh- & Hebevorrichtungen:Haken' \
--text2 'Beauty & Personal Care:Hair Care:Hair Extensions, Wigs & Accessories:Wigs'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tensor([[-3.5115]], device='cuda:0', grad_fn=<AddmmBackward0>)
CPU times: user 37.2 ms, sys: 107 ms, total: 144 ms
Wall time: 9.22 s
